In [1]:
#from langgraph.graph import StateGraph,START,END
from typing import TypedDict,Literal#,Optional,Any #Annotated
#from pydantic import BaseModel,Field

from langchain.tools import BaseTool
from langchain_ollama import ChatOllama
from langchain_groq import ChatGroq
from langchain_core.messages import AIMessage,SystemMessage,HumanMessage,ToolMessage,BaseMessage#FunctionMessage
#from langchain.prompts import MessagesPlaceholder

import subprocess
import os
import locale
import platform

In [2]:
from dotenv import load_dotenv
load_dotenv('C:/env/.env')

True

In [2]:
def get_sys_info():
    sys_info = (
        "###Basic user system information:\n"
        f"Username: {os.getlogin()}\n"
        f"Language: {locale.getlocale()[0]} (some outputs may use this language but answer always in english)\n"
        f"Operational system: {platform.system()} {platform.architecture()[0]}\n"
        f"Terminal: {os.getenv('COMSPEC') if os.name == 'nt' else os.getenv('SHELL')}\n"
        f"You are in the directory: {os.path.abspath('./')} (you can use cd commands to change it)"
        ).replace('\\','\\\\')
    return sys_info

In [3]:
def print_msgs(msgs):
    for msg in msgs: msg.pretty_print()

In [ ]:
class CMDExecTool(BaseTool):
    name: str = 'cmd_exec_tool'
    description: str = ('Execute a CMD prompt command and return the output.\n'
    'Args:\n'
    'cmd: prompt command string to be executed, make sure to include only the commands with correct syntax.')
    request_confirmation: bool = True

    def _run(self, cmd:str):
        cmd_split = cmd.split()
        b_change_dir = False
        if cmd_split[0].lower() == 'cd' and len(cmd_split) == 2:
            cmd = cmd + " && cd"
            b_change_dir = True

        if self.request_confirmation and input(f'Execute command: {cmd}? (y/n)') != 'y':
            return 'canceled by the user'
        
        prompt_result = subprocess.run(cmd, shell=True, capture_output=True,encoding='utf-8',errors='ignore')
        output = prompt_result.stdout.strip() if prompt_result.returncode == 0 else prompt_result.stderr.strip()

        if b_change_dir and prompt_result.returncode == 0:
            try:
                os.chdir(output)
            except Exception as e:
                output = 'error: ' + str(e)
        
        if len(output) > 0 and prompt_result.returncode != 0:
            output = f'Error: {output}'
        if len(output) == 0:
            output = 'success' if prompt_result.returncode == 0 else 'error'

        return output
    

        
class AskUserTool(BaseTool):
    name: str = 'ask_user_tool'
    description: str = (
    'Ask the user for information\n'
    #'Ask the user for preference information or perform necessary tasks.\n'
    #'Usefull to ask relevant information that the user may not included in the input or to ask the user to perform a task.\n'
    #'Avoid asking the user if you can do it by yourself.\n'
    'Args:\n'
    'question: question string to be asked to the user.')

    def _run(self, question):
        return input(question)
        #return f"Question: {question}\nAnswer: {answer}"

class RunPythonCodeTool(BaseTool):
    name: str = 'run_python_code_tool'
    description: str = ('Execute python code.\n'
    'Usefull to execute actions in the user computer or get relevant information.\n'
    "If any information should be returned as output, the return value should be stored in a global variable called 'return_value'\n"
    "Avoid using ; for formatting, instead use \\t and \\n\n"
    "Here is a simple format example using return_value:\n"
    "def sum(x,y):\n"
    "\treturn x+y\n"
    "return_value=sum(3,5)\n"
    'Args:\n'
    'code: python code string, make sure to include only the code with the correct syntax.\n'
    )
    request_confirmation: bool = True

    def _run(self,python_code:str):
        if self.request_confirmation and input('Run code:\n'+python_code) != 'y':
            return None
        locals_dict = {'return_value': None} #locals()
        try:
            exec(python_code,{},locals_dict)
            if locals_dict['return_value'] is None:
                locals_dict['return_value'] = 'Success'
        except Exception as e:
            locals_dict['return_value'] = 'Error: '+ str(e)

        return str( locals_dict['return_value'] )

cmd_exec_tool = CMDExecTool(request_confirmation=True)
ask_user_tool = AskUserTool()
run_python_code_tool = RunPythonCodeTool(request_confirmation=True)
tools = [cmd_exec_tool,ask_user_tool,run_python_code_tool]
tools_dict = {'cmd_exec_tool':cmd_exec_tool,'ask_user_tool':ask_user_tool,'run_python_code_tool':run_python_code_tool}

In [14]:
print(ask_user_tool.description)

Ask the user for preference information or perform necessary tasks.
Usefull to ask relevant information that the user may not included in the input or to ask the user to perform a task.
Args:
question: question string to be asked to the user.


In [30]:
class State(TypedDict):
    user_input:str
    agent_response: AIMessage
    scratchpad:list[BaseMessage]
    final_answer:str

def get_initial_state(user_input):
    return State({'user_input':user_input,'scratchpad':[],'agent_response':None,'final_answer':None})

#llm = ChatGroq(model='llama-3.3-70b-versatile') # qwen/qwen3-32b
llm = ChatOllama(base_url='localhost:11434',model='qwen3:8b',reasoning=False) # qwen3:8b llama3.2:3b
llm_agent=llm.bind_tools(tools)

In [31]:
# def agent_node(state:State):
#     prompt_str = (
#     'You are an agent that solves user computer problems\n'
#     '{sys_info}\n\n'
#     'You must work in a loop where every iteration you will receive the output of the previous iteration.\n'
#     'Once the problem is solved do not pick any tool, instead just give a final answer.\n\n'

#     '###Problem to solve\n{problem}\n\n'

#     "### Input and output of the previous iterations\n\n"
#     '{scratchpad}'
#     ).format(
#         sys_info=get_sys_info(),
#         problem=state['user_input'],
#         scratchpad=state['scratchpad'])
    
#     response = llm_agent.invoke(prompt_str)
#     return response,prompt_str

def agent_node(state:State):
    system_message = (
        'You are an agent that solves user computer problems\n'
        f'{get_sys_info()}\n\n'
        'You must work in a loop where every iteration you will receive the output of the previous iteration.\n'
        # 'Each iteration is sctructured as:\n'
        # 'Action: tool name picked\n' \
        # 'Action Input: the tool input you chosed\n'
        # 'Action Output: The output of the tool\n' \
        'You should use this iterations history to pick your next action\n'
        'Once the problem is solved or failed after many attempts do not pick any tool, instead just give a final answer.'
    )
    messages = [
        SystemMessage(system_message),
        HumanMessage(state['user_input']),
        *state['scratchpad']
    ]
    
    response = llm_agent.invoke(messages)
    return response,messages

In [32]:
state_ = get_initial_state(user_input='Create a folder in my desktop')

In [78]:
res,prompt=agent_node(state_)

In [79]:
res

AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'qwen3:8b', 'created_at': '2025-09-19T13:01:18.9720419Z', 'done': True, 'done_reason': 'stop', 'total_duration': 2887532200, 'load_duration': 212389800, 'prompt_eval_count': 601, 'prompt_eval_duration': 752700900, 'eval_count': 31, 'eval_duration': 1906571000, 'model_name': 'qwen3:8b'}, id='run--658551ef-53fc-42fa-996f-8bd82f2df6d1-0', tool_calls=[{'name': 'cmd_exec_tool', 'args': {'cmd': 'md c:\\Users\\User\\Desktop\\NewFolder'}, 'id': '6aac3627-d51f-4890-a8d9-719384a57f7c', 'type': 'tool_call'}], usage_metadata={'input_tokens': 601, 'output_tokens': 31, 'total_tokens': 632})

In [80]:
res.tool_calls

[{'name': 'cmd_exec_tool',
  'args': {'cmd': 'md c:\\Users\\User\\Desktop\\NewFolder'},
  'id': '6aac3627-d51f-4890-a8d9-719384a57f7c',
  'type': 'tool_call'}]

In [ ]:
print_msgs(prompt)

================================ System Message ================================

You are an agent that solves user computer problems
###Basic user system information:
Username: User
Language: pt_BR (some outputs may use this language but answer always in english)
Operational system: Windows 64bit
Terminal: C:\\Windows\\system32\\cmd.exe
You are in the directory: c:\\Users\\User\\Desktop\\Projects_ML_WEB\\3-ComputerTaskChat\\V-2\\test (you can use cd commands to change it)

You must work in a loop where every iteration you will receive the output of the previous iteration.
You should use this iterations history to pick your next action
Once the problem is solved or failed after many attempts do not pick any tool, instead just give a final answer.
================================ Human Message =================================

Create a folder in my desktop


In [82]:
state_['agent_response'] = res

In [33]:
def tool_condition(state:State) -> Literal['execute_action','__end__']:
    agent_response = state['agent_response']

    if hasattr(agent_response,'tool_calls') and len(agent_response.tool_calls) > 0:
        return 'execute_action'
    
    return '__end__'

In [84]:
tool_condition(state_)

'execute_action'

In [34]:
def execute_action_node(state:State):
    agent_response = state['agent_response']
    scratchpad = state['scratchpad'].copy()
    
    for tool_call in agent_response.tool_calls:
        tool_name = tool_call['name']
        tool_args = tool_call['args']
        tool = tools_dict.get(tool_name,None)
        if tool:
            tool_response = tool.invoke(tool_args)

            ai_msg = AIMessage(content=(
                f'Action: {tool_name}\n'
                f'Action Input: {list(tool_args.values())[0]}\n'
                ))
            tool_msg = ToolMessage(content=tool_response,tool_call_id=tool_call['id'])

            scratchpad += [ai_msg,tool_msg]

            # scratchpad += (f'\n\n###Action: {tool_name}\n'
            #                f'{list(tool_args.values())[0]}\n'
            #                f'###Response\n{tool_response}')
            
    return scratchpad#.strip()

In [ ]:
state_['scratchpad'] = []

In [86]:
scratchpad = execute_action_node(state_)

In [89]:
print_msgs(scratchpad)

================================== Ai Message ==================================

Action: cmd_exec_tool
Action Input: md c:\Users\User\Desktop\NewFolder
================================= Tool Message =================================

success


In [91]:
state_['scratchpad'] = scratchpad

<h1>Simulating the graph flow one by one</h1>
<h2>Iteration 1</h2>

In [39]:
#state_ = get_initial_state(user_input='Write a pdf in my desktop explaining about generative AI in one paragraph.')
state_ = get_initial_state(user_input='Create a folder called Start on my desktop.')

In [40]:
res,prompt=agent_node(state_)
state_['agent_response'] =res

In [41]:
res.tool_calls

[{'name': 'cmd_exec_tool',
  'args': {'cmd': 'mkdir C:\\Users\\User\\Desktop\\Start'},
  'id': '677f9cc0-077a-41b3-910a-1e481124ead8',
  'type': 'tool_call'}]

In [18]:
print(res.tool_calls[0]['args']['python_code'])

import os
def generate_pdf():
	from fpdf import FPDF
	
	# Create a PDF object
	pdf = FPDF()
	
	# Add a page
	pdf.add_page()
	
	# Set font
	pdf.set_font("Arial", size=12)
	
	# Add a paragraph about generative AI
	pdf.multi_cell(0, 10, "Generative AI refers to artificial intelligence systems that can create new content, such as text, images, audio, and video, based on patterns learned from existing data. These systems are powered by advanced machine learning algorithms, particularly deep learning models like neural networks, which enable them to generate highly realistic and contextually relevant outputs. Generative AI has a wide range of applications, including content creation, design, entertainment, and research, and it continues to evolve with ongoing advancements in technology.")
	
	# Save the PDF to the desktop
	pdf.output("C:\\Users\\User\\Desktop\\generative_ai.pdf")

	return_value = "PDF created successfully on the desktop."

generate_pdf()


In [19]:
print_msgs(prompt)

================================ System Message ================================

You are an agent that solves user computer problems
###Basic user system information:
Username: User
Language: pt_BR (some outputs may use this language but answer always in english)
Operational system: Windows 64bit
Terminal: C:\\Windows\\system32\\cmd.exe
You are in the directory: c:\\Users\\User\\Desktop\\Projects_ML_WEB\\3-ComputerTaskChat\\V-2\\test (you can use cd commands to change it)

You must work in a loop where every iteration you will receive the output of the previous iteration.
You should use this iterations history to pick your next action
Once the problem is solved or failed after many attempts do not pick any tool, instead just give a final answer.
================================ Human Message =================================

Write a pdf in my desktop explaining about generative AI in one paragraph.


In [20]:
tool_condition(state_)

'execute_action'

In [21]:
scratchpad = execute_action_node(state_)
state_['scratchpad'] = scratchpad

In [22]:
print_msgs(scratchpad)

================================== Ai Message ==================================

Action: run_python_code_tool
Action Input: import os
def generate_pdf():
	from fpdf import FPDF
	
	# Create a PDF object
	pdf = FPDF()
	
	# Add a page
	pdf.add_page()
	
	# Set font
	pdf.set_font("Arial", size=12)
	
	# Add a paragraph about generative AI
	pdf.multi_cell(0, 10, "Generative AI refers to artificial intelligence systems that can create new content, such as text, images, audio, and video, based on patterns learned from existing data. These systems are powered by advanced machine learning algorithms, particularly deep learning models like neural networks, which enable them to generate highly realistic and contextually relevant outputs. Generative AI has a wide range of applications, including content creation, design, entertainment, and research, and it continues to evolve with ongoing advancements in technology.")
	
	# Save the PDF to the desktop
	pdf.output("C:\\Users\\User\\Desktop\\generativ

<h2>Iteration 2</h2>

In [23]:
res,prompt=agent_node(state_)
state_['agent_response'] =res

In [24]:
res.tool_calls

[]

In [26]:
print(res.content)

The PDF explaining about generative AI has been successfully created on your desktop. You can find it at `C:\\Users\\User\\Desktop\\generative_ai.pdf`. Let me know if you need anything else!


In [27]:
print_msgs(prompt)

================================ System Message ================================

You are an agent that solves user computer problems
###Basic user system information:
Username: User
Language: pt_BR (some outputs may use this language but answer always in english)
Operational system: Windows 64bit
Terminal: C:\\Windows\\system32\\cmd.exe
You are in the directory: c:\\Users\\User\\Desktop\\Projects_ML_WEB\\3-ComputerTaskChat\\V-2\\test (you can use cd commands to change it)

You must work in a loop where every iteration you will receive the output of the previous iteration.
You should use this iterations history to pick your next action
Once the problem is solved or failed after many attempts do not pick any tool, instead just give a final answer.
================================ Human Message =================================

Write a pdf in my desktop explaining about generative AI in one paragraph.
================================== Ai Message ==================================

Acti

In [28]:
tool_condition(state_)

'__end__'